## Notebook 02 — Data Cleaning & Preprocessing
#### This notebook performs cleaning operations on the Facebook Ads dataset: duplicates, missing values, datatypes, and column formatting.

In [1]:
import pandas as pd
import numpy as np
import os
import sys

# Ensure project root is accessible
project_root = os.path.abspath("..")
sys.path.append(project_root)

from src.data_cleaning import remove_duplicates, handle_missing_values

In [2]:
input_path = "../outputs/intermediate_files/raw_ingested_data.csv"

df = pd.read_csv(input_path)
df.head()

,campaign_name,adset_name,date,spend,impressions,clicks,ctr,purchases,revenue,roas,creative_type,creative_message,audience_type,platform,country
0,Men ComfortMax Launch,Adset-1 Retarget,2025-01-01,640.09,235597,4313.0,0.0183,80,1514.28,2.37,Image,Breathable organic cotton that moves with you ...,Broad,Facebook,US
1,Men ComfortMax Launch,Adset-1 Retarget,2025-01-02,373.75,276194,5429.0,0.0197,94,4152.81,11.11,Video,No ride‑up guarantee — best‑selling men briefs...,Broad,Facebook,US
2,Men ComfortMax Launch,Adset-1 Retarget,2025-01-03,703.79,466572,9830.0,0.0211,240,4893.43,6.95,UGC,Cooling mesh panels for workouts — men boxers ...,Broad,Facebook,US
3,Men_ComfortMax_Launch,Adset-1 Retarget,2025-01-04,441.50,193230,2424.0,0.0125,61,1330.10,3.01,Image,Cooling mesh panels for workouts — men athleti...,Lookalike,Facebook,US
4,Men ComfortMax Launch,Adset-1 Retarget,2025-01-05,NaN,404300,6372.0,0.0158,181,7027.20,51.52,Video,Summer‑ready essentials — sweat‑wicking men at...,Lookalike,Instagram,IN


In [3]:
df.shape

(4500, 15)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   campaign_name     4500 non-null   object 
 1   adset_name        4500 non-null   object 
 2   date              4500 non-null   object 
 3   spend             4387 non-null   float64
 4   impressions       4500 non-null   int64  
 5   clicks            4348 non-null   float64
 6   ctr               4500 non-null   float64
 7   purchases         4500 non-null   int64  
 8   revenue           4367 non-null   float64
 9   roas              4491 non-null   float64
 10  creative_type     4500 non-null   object 
 11  creative_message  4500 non-null   object 
 12  audience_type     4500 non-null   object 
 13  platform          4500 non-null   object 
 14  country           4500 non-null   object 
dtypes: float64(5), int64(2), object(8)
memory usage: 527.5+ KB


In [5]:
df.isnull().sum()

campaign_name         0
adset_name            0
date                  0
spend               113
impressions           0
clicks              152
ctr                   0
purchases             0
revenue             133
roas                  9
creative_type         0
creative_message      0
audience_type         0
platform              0
country               0
dtype: int64

In [6]:
df = remove_duplicates(df)
print("Duplicates removed.")
df.shape

Duplicates removed.


(4500, 15)

In [7]:
df = handle_missing_values(df, method="drop")
df.isnull().sum()

campaign_name       0
adset_name          0
date                0
spend               0
impressions         0
clicks              0
ctr                 0
purchases           0
revenue             0
roas                0
creative_type       0
creative_message    0
audience_type       0
platform            0
country             0
dtype: int64

In [15]:
numeric_columns = [
    "spend",
    "impressions",
    "clicks",
    "ctr",
    "purchases",
    "revenue",
    "roas"
]

In [16]:
df.columns

Index(['campaign_name', 'adset_name', 'date', 'spend', 'impressions', 'clicks',
       'ctr', 'purchases', 'revenue', 'roas', 'creative_type',
       'creative_message', 'audience_type', 'platform', 'country', 'cpc',
       'cpm'],
      dtype='object')

In [17]:
df = df.dropna(subset=numeric_columns, how="any")
df.shape

(4116, 17)

In [19]:
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [20]:
df["cpc"] = df["spend"] / df["clicks"]
df["cpm"] = (df["spend"] * 1000) / df["impressions"]
df["revenue_per_purchase"] = df["revenue"] / df["purchases"]

In [21]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)
df.head()

,campaign_name,adset_name,date,spend,impressions,clicks,ctr,purchases,revenue,roas,creative_type,creative_message,audience_type,platform,country,cpc,cpm,revenue_per_purchase
0,Men ComfortMax Launch,Adset-1 Retarget,2025-01-01,640.09,235597,4313.0,0.0183,80,1514.28,2.37,Image,Breathable organic cotton that moves with you ...,Broad,Facebook,US,0.148409,2.716885,18.928500
1,Men ComfortMax Launch,Adset-1 Retarget,2025-01-02,373.75,276194,5429.0,0.0197,94,4152.81,11.11,Video,No ride‑up guarantee — best‑selling men briefs...,Broad,Facebook,US,0.068843,1.353215,44.178830
2,Men ComfortMax Launch,Adset-1 Retarget,2025-01-03,703.79,466572,9830.0,0.0211,240,4893.43,6.95,UGC,Cooling mesh panels for workouts — men boxers ...,Broad,Facebook,US,0.071596,1.508427,20.389292
3,Men_ComfortMax_Launch,Adset-1 Retarget,2025-01-04,441.50,193230,2424.0,0.0125,61,1330.10,3.01,Image,Cooling mesh panels for workouts — men athleti...,Lookalike,Facebook,US,0.182137,2.284842,21.804918
5,Men Comfortmax Launch,Adset-1 Retarget,2025-01-06,579.03,180096,2356.0,0.0131,41,1545.67,2.67,Image,Breathable bamboo that moves with you — limite...,Lookalike,Instagram,UK,0.245768,3.215119,37.699268


In [22]:
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

for col in ["cpc", "cpm", "ctr", "roas"]:
    if col in df.columns:
        df = remove_outliers(df, col)

In [23]:
output_path = "../outputs/intermediate_files/cleaned_data.csv"
df.to_csv(output_path, index=False)

print("Cleaned dataset saved to:", output_path)

Cleaned dataset saved to: ../outputs/intermediate_files/cleaned_data.csv
